In [1]:
!pip install psycopg2-binary pgvector

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 26.4 MB/s  0:00:00

   -------------------- ------------------- 1/2 [pgvector]
   ---------------------------------------- 2/2 [pgvector]



In [3]:
database="zf_db"
user="zf_admin"
password="qnwkehlwk9090"

import psycopg2
from pgvector.psycopg2 import register_vector
import numpy as np

# DB 연결 정보 (이전과 동일)
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database=database,
    user=user,
    password=password
)
conn.autocommit = True 
register_vector(conn) 
cur = conn.cursor()

try:
    print("🎬 영화 검색 예제 설정 시작...")
    
    # 1. 테이블 생성 (영화 제목, 줄거리, 3차원 특징 벡터)
    cur.execute("DROP TABLE IF EXISTS movies")
    cur.execute("CREATE TABLE movies (id bigserial PRIMARY KEY, title text, summary text, embedding vector(3))")
    
    # 2. 영화 데이터 입력 (벡터는 가상의 '액션/로맨스/코미디' 점수로 가정)
    movie_data = [
        ('미션 임파서블', '첩보 요원 이단 헌트의 목숨 건 액션 대작.', [1.0, 0.1, 0.2]), # 액션 높음
        ('러브 액츄얼리', '크리스마스를 배경으로 한 여러 커플의 로맨틱 코미디.', [0.1, 0.9, 0.7]), # 로맨스/코미디 높음
        ('올드 보이', '감금된 남자의 복수극. 무거운 드라마와 폭력적인 액션.', [0.6, 0.1, 0.0]), # 액션/드라마 높음
        ('나 홀로 집에', '어린 아이가 도둑들을 물리치는 코미디 영화.', [0.3, 0.1, 1.0]), # 코미디 높음
    ]
    
    for title, summary, embedding in movie_data:
        cur.execute("INSERT INTO movies (title, summary, embedding) VALUES (%s, %s, %s)", 
                    (title, summary, embedding))
    
    print(f"✅ 영화 데이터 {len(movie_data)}개 저장 완료.")

    # 3. 검색 쿼리 실행 (액션과 로맨스 중간쯤 되는 영화를 찾고 싶다면?)
    # 검색 벡터: [0.7, 0.7, 0.0] -> 액션 0.7, 로맨스 0.7, 코미디 0.0
    query_vector = [0.7, 0.7, 0.0] 
    
    # Cosine Distance 연산자 <=> 를 사용합니다. 거리가 0에 가까울수록 유사합니다.
    cur.execute("""
        SELECT 
            title, 
            summary, 
            embedding <=> %s::vector AS distance 
        FROM movies 
        ORDER BY distance 
        LIMIT 3
    """, (query_vector,))
    
    results = cur.fetchall()
    
    print(f"\n🔍 검색 벡터 '{query_vector}' (액션/로맨스) 결과:")
    for title, summary, distance in results:
        print(f"--------------------------------------------------")
        print(f"제목: {title}")
        print(f"줄거리: {summary}")
        print(f"코사인 거리(유사도): {distance:.4f} (0에 가까울수록 유사)")

except Exception as e:
    print(f"❌ 에러 발생: {e}")

finally:
    if conn:
        cur.close()
        conn.close()
        print("\n🔒 DB 연결이 종료되었습니다.")

🎬 영화 검색 예제 설정 시작...
✅ 영화 데이터 4개 저장 완료.

🔍 검색 벡터 '[0.7, 0.7, 0.0]' (액션/로맨스) 결과:
--------------------------------------------------
제목: 올드 보이
줄거리: 감금된 남자의 복수극. 무거운 드라마와 폭력적인 액션.
코사인 거리(유사도): 0.1863 (0에 가까울수록 유사)
--------------------------------------------------
제목: 미션 임파서블
줄거리: 첩보 요원 이단 헌트의 목숨 건 액션 대작.
코사인 거리(유사도): 0.2409 (0에 가까울수록 유사)
--------------------------------------------------
제목: 러브 액츄얼리
줄거리: 크리스마스를 배경으로 한 여러 커플의 로맨틱 코미디.
코사인 거리(유사도): 0.3822 (0에 가까울수록 유사)

🔒 DB 연결이 종료되었습니다.
